In [1]:
import numpy as np

def cosine_sim(x,y):
    num = np.dot(x, y)
    denom = np.linalg.norm(x) * np.linalg.norm(y)
    return num / denom

In [35]:
from gensim.models.word2vec import Word2Vec

modeldir = '../../alignment/1-year/model/'
models = []

for i in range(1946, 2024):
    year = modeldir + str(i) + '.model'
    model = Word2Vec.load(year)
    models.append(model)

In [39]:
search_word = models[0].wv['杜鲁门']

year = 1946
for model in models:
    repr = str(year) + ": "
    for pair in model.wv.most_similar([search_word], topn=10):
        repr += pair[0] + '\t'
    print(repr)
    year += 1

1946: 杜鲁门	总统	华莱士	外交	华氏	贝纳斯	德拉西	泼贝尔	白宫	华盛顿	
1947: 艾德礼	杜鲁门	下院	保守党	外交	范登堡	贝纳斯	华氏	贝文	众议院	
1948: 麦氏	国务院	杜鲁门	下院	艾德礼	备忘录	修正案	折衷	众院	贝纳斯	
1949: 杜鲁门	康纳利	报界	外委会	国务院	参院	华莱士	美国	范登堡	评称	
1950: 杜鲁门	力主	劳工党	宣称	艾德礼	下院	德田	贝文	外委会	对意	
1951: 众议院	议员们	苏美	泰晤士报	参议院	众院	新阁	杜鲁门	邱吉尔	艾德礼	
1952: 康纳利	申明	艾德礼	泰晤士报	盟国	议员们	邱吉尔	国务院	组阁	保守党	
1953: 艾德礼	葛罗米柯	国务院	波立特	保守党	史密斯	盟国	宣称	通牒	杜威	
1954: 保守党	训令	下院	声明	邱吉尔	政纲	罗斯福	敦促	抗议书	断然	
1955: 允诺	邱吉尔	戴高乐	威尔逊	艾德礼	杜鲁门	官方	宣战	反对派	诺言	
1956: 杜鲁门	波立特	福斯特	申明	白宫	艾德礼	参院	声明	众议院	报界	
1957: 参院	艾德礼	杜鲁门	盟国	报界	时报	现政府	美联社	威尔逊	路透社	
1958: 白皮书	议员们	史密斯	威尔逊	旧金山	众议院	工业家	白宫	外交	艾德礼	
1959: 中止	美联社	杜鲁门	施政	保守党	参院	美国	决议案	宣称	纽约州	
1960: 保守党	继任	任用	下院	决议案	麦克阿瑟	施政	声称	各报	拟议	
1961: 施政	保守党	国大党	现政府	扬言	参院	参议院	史密斯	路透社	断然	
1962: 白宫	倒台	参院	反省	组阁	下院	大加	美国	允诺	参议员	
1963: 现政府	杜鲁门	两院	参议院	声称	美国	社会党人	自由党	倒台	强硬	
1964: 惩处	诺言	威尔逊	给与	慕尼黑	工党	有罪	英共	拟议	福斯特	
1965: 参院	声言	保守党	朝日	泰晤士报	显而易见	罗斯福	众议院	草拟	非正式	
1966: 白皮书	福斯特	参院	威尔逊	复会	尽人皆知	下院	决议案	国务卿	修正案	
1967: 下院	此次	观察家	安理会	众议院	工党	抨击	期中	塔斯社	提案	
1968: 议程	威尔逊	诺言	声称	民权	提案	生效	抨击	责成	抱怨	
1969: 观察家	好战	强硬	就此	备忘录	